<a href="https://colab.research.google.com/github/Fabiorsi1979/DS-Comunidade/blob/main/SQL_avancado_Windows_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Windows Function - Agregação
##### AVG () , MAX (), MIN () e SUM ()

###### Sempre que for necessário + que uma agregação na mesma consulta, a WINDOWS FUNCTION possilita que com poucas linhas de código seja possível a análise.
~~~ sql
  SELECT
      <column 1>,
      <column 2>,
      <windows function (SUM, AVG, MAX ou MIN)> (expression) OVER (PARTITTION BY <partition list>
                                                              ORDER BY <order list>
                                                              ROWS frame_cause)
  FROM <table>
~~~

# SQL Windows Function - Funcões de Ranqueamento

##### ROW_NUMBER () = Mostra o nº da linha até o valor da última linha, não leva em consideração valores iguais, portanto o mesmo valor pode ter posição 11,12,13, n.vezes dependendo do número de vezes que se repete.
##### RANK () = O Rank agrupa na mesma posição os numeros iguais, porém pula a sequência (exemplo abaixo).
##### DENSE_RANK () = Também agrupa as posições iguais mas não pula a sequência
##### PERCENT_RANK () = Apresenta o ranqueamento em %, ou seja, até determinada posição qual % representa de determinada tabela.
##### NTILE () = Separa em blocos onde é possivel determinar a sua divisão podendo ser 4 partes de 25%, 5 partes de 20%, 3 partes de 33,33%, etc...
~~~ sql
 Exemplos:
      PRICE - ROW_NUMBER - RANK - DENSE_RANK - PERCENT_RANK - NTILE
      44,97 -    1       -  1   -      1     -       0,0%   -   1
      45,98 -    2       -  2   -      2     -      40,0%   -   1
      45,98 -    3       -  2   -      2     -      60,0%   -   2
      46,99 -    4       -  4   -      3     -      80,0%   -   2
      51,82 -    5       -  5   -      4     -     100,0%   -   3
~~~
# SQL Windows Function - Funcões de Valor

##### LAG () = desloca a linha em uma posição para baixo
##### LEAD () = retira a primeira linha deslocando para cima as demais
##### FIRST_VALUE () = retorna o menor valor dentro do segmento e replica para todas as linhas.
##### LAST_VALUE () = retorna o último valor dentro do segmento e replica para toda as linmhas
##### NTH_VALUE (<column>, x) = replica os valores da coluna informada a partir da posição solicitada.
~~~ sql
 Exemplos:
                                                   NHT_VALUE ( DATA, 3 ) ...    
   DATA   -   LAG    -   LEAD    - FIRST_VALUE - LAST_VALUE - NTH_VALUE
 11/05/19 -          -  12/05/19 -  11/05/2019 -  15/05/19  -  
 12/05/19 - 11/05/19 -  13/05/19 -  11/05/2019 -  15/05/19  -  
 13/05/19 - 12/05/19 -  14/05/19 -  11/05/2019 -  15/05/19  -  13/05/19
 14/05/19 - 13/05/19 -  15/05/19 -  11/05/2019 -  15/05/19  -  14/05/19
 15/05/19 - 14/05/19 -  16/05/19 -  11/05/2019 -  15/05/19  -  15/05/19
~~~

# SQL Windows Function - Controle tamanho da JANELA

##### PRECEDING -> Quantidade de linhas que precede o registro atual, precisa informar o número.
##### CURRENT -> Significa registro atual
##### FOLLOWING -> Quantidade de linhas após o registro atual, precisa informar o número.
##### UNBOUNDED->  Pode ser usado tanto para antes quanto para após o registro atual, significa toda a linha histórica (desde o início ou até o último)

~~~ sql
  ______________________________________________________________________
  ^                    ^           ^          ^                        ^
UNBOUNDED         PRECEDING     CURRENT    FOLLOWING           UNBOUNDED

# exemplo

SELECT
    o.order_purchase_timestamp,
    oi.price,
    AVG ( oi.price ) OVER ( ORDER BY o.order_purchase_timestamp ASC
                            ROWS BETWEEN UNBOUNDED PRECEDING AND
                            ROWS UNBOUNDED FOLLOWING ) AS mediaMovel
FROM orders o INNER JOIN order_items oi ON ( oi.order_id = o.order_id )

# exemplo

SELECT
    o.order_purchase_timestamp,
    oi.price,
    AVG ( oi.price ) OVER ( ORDER BY o.order_purchase_timestamp ASC
                            ROWS 7 PRECEDING AND
                            ROWS 5 FOLLOWING ) AS mediaMovel
FROM orders o INNER JOIN order_items oi ON ( oi.order_id = o.order_id )
~~~


In [3]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [4]:
path= '/content/drive/MyDrive/db_olist.sqlite'

In [5]:
import sqlite3
import pandas as pd

In [6]:
def consulta_WFA(*args):
  conexao = sqlite3.connect(path)
  cursor = conexao.cursor()

  cursor.execute(consulta)
  dados = cursor.fetchall()

  cursor.close()
  conexao.close()

  consulta_sql = pd.DataFrame(data=dados)
  return consulta_sql

#### 1. Cria uma consulta que liste a categoria, o preço do produto e classificação de cada produto dentro da sua categoria do mais caro ao mais barato com base no preço no seu preço médio, a partir do dia 01 de Junho de 2018.

In [5]:
consulta ="""
SELECT
categoria,
product_id,
preco,
DENSE_RANK() OVER ( PARTITION BY categoria ORDER BY preco DESC ) AS classificacao
FROM (SELECT
        p.product_id,
        p.product_category_name AS categoria,
        AVG( oi.price ) AS preco
      FROM orders o LEFT JOIN order_items oi ON ( oi.order_id = o.order_id )
                    LEFT JOIN products p ON ( p.product_id = oi.product_id )
                    LEFT JOIN order_reviews or2 ON ( or2.order_id = o.order_id )
      WHERE o.order_purchase_timestamp > '2018-06-01 00:00:00'
      GROUP BY p.product_id
      HAVING p.product_category_name IS NOT NULL)
LIMIT 10
"""
resultado = consulta_WFA(consulta)
resultado.columns = ["Categoria","Codigo Produto","Preço","Classificação"]
resultado

,Categoria,Codigo Produto,Preço,Classificação
0,agro_industria_e_comercio,eddb814fb553b6951f51b34c5f578ba0,1899.00,1
1,agro_industria_e_comercio,ea1d59339533cb7f68f0319501d8e4d4,1234.00,2
2,agro_industria_e_comercio,c89226b8a795ae3d6bca9d90b20dbf04,940.50,3
3,agro_industria_e_comercio,e38487b365f562f0f5f6d8362cd36593,699.98,4
4,agro_industria_e_comercio,b5015f9057f27b5bdefde4ca99368b54,559.99,5
5,agro_industria_e_comercio,b8faf829f47ba1f35f169da4acbdb88d,549.00,6
6,agro_industria_e_comercio,f72b26eeffe2345b486cfeb723305bac,498.00,7
7,agro_industria_e_comercio,66e94c5c07d64d44c16cd6903a79a453,495.60,8
8,agro_industria_e_comercio,49b6d7e8192cd04ef8f69b1bf00b7eb1,469.00,9
9,agro_industria_e_comercio,423a6644f0aa529e8828ff1f91003690,455.60,10


#### 2.  Crie uma consulta que exiba a data de compra, o valor de cada venda e o total acumulado de vendas até aquela data

In [ ]:
consulta ="""
SELECT
    o.order_purchase_timestamp,
    op.payment_value,
    SUM ( op.payment_value ) OVER ( ORDER BY o.order_purchase_timestamp DESC) acumuladoVenda
FROM orders o LEFT JOIN order_payments op ON ( op.order_id = o.order_id )
WHERE op.payment_value IS NOT NULL
LIMIT 10
"""
resultado = consulta_WFA(consulta)
resultado.columns = ["Data Venda","Valor Pagamento","Acumulado"]
resultado

,Data Venda,Valor Pagamento,Acumulado
0,2018-10-17 17:30:18,89.71,89.71
1,2018-10-16 20:16:02,222.03,311.74
2,2018-10-03 18:55:29,197.55,509.29
3,2018-10-01 15:30:09,80.38,589.67
4,2018-09-29 09:13:03,137.03,726.70
5,2018-09-26 08:40:15,94.63,821.33
6,2018-09-25 11:59:18,221.54,1042.87
7,2018-09-20 13:54:16,191.46,1234.33
8,2018-09-17 17:21:16,84.58,1318.91
9,2018-09-13 09:56:12,414.04,1732.95


#### 3. Crie uma consulta que exiba a data de compra, o valor de cada venda e a média móvel dos últimos três valores de venda incluindo o valor atual

In [ ]:
consulta ="""
SELECT
    o.order_approved_at,
    op.payment_value,
    AVG ( op.payment_value ) OVER ( ORDER BY o.order_approved_at DESC
                                    ROWS BETWEEN 3 PRECEDING AND CURRENT ROW ) MediaMovel
FROM orders o LEFT JOIN order_payments op ON ( op.order_id = o.order_id )
LIMIT 10
"""
resultado = consulta_WFA(consulta)
resultado.columns = ["Data Aprovação Compra","Valor da Compra","Média móvel ult. 3 dias"]
resultado

,Data Aprovação Compra,Valor da Compra,Média móvel ult. 3 dias
0,2018-09-03 17:40:06,166.46,166.460000
1,2018-08-29 15:10:26,93.75,130.105000
2,2018-08-29 15:05:22,33.23,97.813333
3,2018-08-29 14:31:07,510.96,201.100000
4,2018-08-29 14:30:23,61.29,174.807500
5,2018-08-29 12:35:17,73.10,169.645000
6,2018-08-29 11:24:02,119.42,191.192500
7,2018-08-29 10:35:16,14.29,67.025000
8,2018-08-29 10:04:16,178.25,96.265000
9,2018-08-29 09:25:12,106.95,104.727500


#### 4. Crie uma consulta que lista a data da compra, o número de produtos vendidos e o crescimento das vendas com relação ao dia anterior.

In [ ]:
consulta ="""
SELECT
    dataVenda,
    produtosVendidos,
    vendasDia - LAG ( vendasDia ) OVER ( ORDER BY dataVenda DESC
                              ROWS 1 PRECEDING ) comparativo_Dia_anterior
FROM ( SELECT
            DATE( o.order_purchase_timestamp ) AS dataVenda,
            COUNT ( oi.product_id ) AS produtosVendidos,
            SUM ( op.payment_value ) OVER ( PARTITION BY DATE (o.order_purchase_timestamp)
                                            ORDER BY DATE (o.order_purchase_timestamp) DESC) AS vendasDia
        FROM orders o LEFT JOIN order_items oi ON ( oi.order_id = o.order_id )
                      LEFT JOIN order_payments op ON ( op.order_id = o.order_id )
        GROUP BY DATE( o.order_purchase_timestamp )
        HAVING op.payment_value  IS NOT NULL
        LIMIT 10 ) AS tabVendas;
"""
resultado = consulta_WFA(consulta)
resultado.columns = ["Data Compra","Qtde Produtos vendidos","Crescimento das Vendas"]
resultado

,Data Compra,Qtde Produtos vendidos,Crescimento das Vendas
0,2016-10-08,46,NaN
1,2016-10-07,53,117.11
2,2016-10-06,58,-129.66
3,2016-10-05,60,247.91
4,2016-10-04,76,-188.33
5,2016-10-03,8,-70.56
6,2016-10-02,1,68.39
7,2016-09-13,0,-68.39
8,2016-09-05,1,34.11
9,2016-09-04,2,61.17


#### 5. Crie uma consulta que exiba o estado do cliente, a categoria, a quantidade de produtos vendidos e o percentual de vendas em relação ao total vendido no estado.


##### primeira parte da resposta

In [7]:
consulta ="""
SELECT
  o.order_id AS pedido,
  o.customer_id AS cliente,
  op.payment_value AS vendas,
  p.product_category_name AS categoria,
  c.customer_state AS UF,
  COUNT ( oi.product_id ) Qtde_vendas,
  SUM ( op.payment_value ) OVER ( PARTITION BY c.customer_state ) AS totalVendasUF,
  SUM ( op.payment_value ) OVER ( PARTITION BY o.customer_id ) AS totalVendasCliente
FROM orders o INNER JOIN order_payments op ON ( op.order_id = o.order_id )
              INNER JOIN order_items oi    ON ( oi.order_id = o.order_id )
              INNER JOIN products p        ON ( p.product_id = oi.product_id )
              INNER JOIN customer c        ON ( c.customer_id = o.customer_id )
GROUP BY cliente
"""
resultado = consulta_WFA(consulta)
resultado.columns = ["Código Pedido","Código Cliente","Valor pago","Categoria","Uf cliente","Qtde Vendaq","Total vendas UF","Total vendas Cliente"]
resultado

,Código Pedido,Código Cliente,Valor pago,Categoria,Uf cliente,Qtde Vendaq,Total vendas UF,Total vendas Cliente
0,7fdefbf564eb66830407026a86005ce2,013bdb994a9c8f09fde3f5f543e698ad,248.71,esporte_lazer,AC,1,18981.89,248.71
1,40aecb2022cf80ba1efc9b0ddf5cd690,061a531d906f4bfb762be36757e41f7b,82.42,perfumaria,AC,1,18981.89,82.42
2,ce720cfa1cadab0ab4c19a956254d382,08420efd8b1c0be27393032e21a993ee,77.20,None,AC,1,18981.89,77.20
3,b701057e29518346f3b457eaddf5c0ba,0f32385df13e46d88d997460208bc866,94.37,informatica_acessorios,AC,1,18981.89,94.37
4,a7abe735fa002a72636ab38fcac70330,10ad09201fcc1c82d181ff7234bcdb3b,62.92,informatica_acessorios,AC,1,18981.89,62.92
...,...,...,...,...,...,...,...,...
98660,8e79eeb59f5108d06a807743a3482bfb,fbfebde26d0e73345094b962c8209a1e,65.62,telefonia,TO,1,59676.56,65.62
98661,747674a4b8b96ff3a901d54e528c9059,fc3e5e2cdea306080a59bea96d94f4b1,112.10,relogios_presentes,TO,1,59676.56,112.10
98662,e90fcc2b428a70a691e3b919ec4d8726,fdbb5e86252d6bb9bfddaabaf1ff7f56,155.70,moveis_decoracao,TO,2,59676.56,155.70
98663,037d59208a513ace896d132dc8c8a93d,fe9e048ae4267835d2658aca15096833,563.07,bebes,TO,1,59676.56,563.07


## Resposta final ao Exercício 5

In [10]:
#Crie uma consulta que exiba o estado do cliente, a categoria,
#a quantidade de produtos vendidos e o percentual de vendas em relação ao total vendido no estado
consulta ="""
SELECT
      tabUF.UF,
      tabUF.cliente,
      tabUF.categoria,
      tabUF.Qtde_vendas,
      ((tabUF.totalVendasCliente/ tabUF.totalVendasUF)*100) AS percentVendas
FROM (  SELECT
          o.order_id AS pedido,
          o.customer_id AS cliente,
          op.payment_value AS vendas,
          p.product_category_name AS categoria,
          c.customer_state AS UF,
          COUNT ( oi.product_id ) Qtde_vendas,
          SUM ( op.payment_value ) OVER ( PARTITION BY c.customer_state ) AS totalVendasUF,
          SUM ( op.payment_value ) OVER ( PARTITION BY o.customer_id ) AS totalVendasCliente
        FROM orders o INNER JOIN order_payments op ON ( op.order_id = o.order_id )
                      INNER JOIN order_items oi    ON ( oi.order_id = o.order_id )
                      INNER JOIN products p        ON ( p.product_id = oi.product_id )
                      INNER JOIN customer c        ON ( c.customer_id = o.customer_id )
        GROUP BY cliente) AS tabUF
GROUP BY tabUF.UF, tabUF.cliente, tabUF.categoria
"""
resultado = consulta_WFA(consulta)
resultado.columns = ["UF cliente","Código Cliente","Categoria compra","Qtde comprada","% Percentual"]
resultado

,UF cliente,Código Cliente,Categoria compra,Qtde comprada,% Percentual
0,AC,013bdb994a9c8f09fde3f5f543e698ad,esporte_lazer,1,1.310249
1,AC,061a531d906f4bfb762be36757e41f7b,perfumaria,1,0.434203
2,AC,08420efd8b1c0be27393032e21a993ee,None,1,0.406703
3,AC,0f32385df13e46d88d997460208bc866,informatica_acessorios,1,0.497158
4,AC,10ad09201fcc1c82d181ff7234bcdb3b,informatica_acessorios,1,0.331474
...,...,...,...,...,...
98660,TO,fbfebde26d0e73345094b962c8209a1e,telefonia,1,0.109959
98661,TO,fc3e5e2cdea306080a59bea96d94f4b1,relogios_presentes,1,0.187846
98662,TO,fdbb5e86252d6bb9bfddaabaf1ff7f56,moveis_decoracao,2,0.260906
98663,TO,fe9e048ae4267835d2658aca15096833,bebes,1,0.943536
